In [ ]:
import pandas as pd

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 20)
pd.plotting.register_matplotlib_converters()

import numpy as np
import seaborn as sns
import missingno as mgn

import plotly.express as px

In [ ]:
df = pd.read_csv('../data/full_data.csv')
df

In [ ]:
page106523 = df.query('page_id == 106523')

1. Concat the authors by `page_id`, `day`, `publish_date`, `word_count`

In [ ]:
pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.max_row', 139)

In [ ]:
page106523_aggr = page106523[['page_id', 'date', 'publish_date', 'word_count', 'authors']]
page106523_aggr.loc[:, 'authors'] = page106523_aggr.authors.apply(lambda s: s.replace(' und ', ';'))

page106523_aggr = page106523_aggr[['page_id', 'date', 'publish_date', 'word_count', 'authors']]\
    .groupby(['page_id', 'date', 'publish_date', 'word_count']) \
        .agg(lambda auth: ';'.join(auth))

page106523_aggr = page106523_aggr.authors.apply(lambda auths: sorted(list(set(auths.split(';'))))).to_frame()

It worked!

In [ ]:
df_aggr = df[['page_id', 'date', 'publish_date', 'word_count', 'authors']]
df_aggr.loc[:, 'authors'] = df_aggr.authors.apply(lambda s: s.replace(' und ', ';'))

df_aggr = df_aggr[['page_id', 'date', 'publish_date', 'word_count', 'authors']]\
    .groupby(['page_id', 'date', 'publish_date', 'word_count']) \
        .agg(lambda auth: ';'.join(auth))

df_aggr = df_aggr.authors.apply(lambda auths: sorted(list(set(auths.split(';'))))).to_frame()

In [ ]:
df = pd.merge(df.drop(['authors'], axis=1), 
        df_aggr,
        on=['page_id', 'date', 'publish_date', 'word_count'], how='left')

In [ ]:
page1014826 = df.query('page_id == 1014826')

Now recalculate the versions

In [ ]:
df.loc[:, 'authors'] = df.loc[:, 'authors'].apply(lambda l: str(l))

In [ ]:
#df.loc[:, 'authors'] = df.authors.str.removeprefix("['")
#df.loc[:, 'authors'] = df.authors.str.removesuffix("']")

### This should do the same thing!
df.authors.apply(''.join)

In [ ]:
temp = df[['page_id', 'word_count', 'publish_date', 'authors']].copy()
temp.drop_duplicates(inplace=True)

In [ ]:
temp = temp.drop_duplicates()
temp = temp.sort_values('publish_date')

wc_versions = temp.groupby('page_id')['word_count'].transform(lambda x: pd.factorize(x)[0])
publish_versions = temp.groupby('page_id')['publish_date'].transform(lambda x: pd.factorize(x)[0])
authors_versions = temp.groupby('page_id')['authors'].transform(lambda x: pd.factorize(x)[0])

version_count = 10000*wc_versions + 1000*publish_versions + 1*authors_versions
temp['ver_id_wc'] = wc_versions
temp['ver_id_pub'] = publish_versions
temp['ver_id_auth'] = authors_versions
temp['version_id_raw'] = version_count
temp['version_id_new'] = temp.groupby('page_id')['version_id_raw'].transform(lambda x: pd.factorize(x)[0])

df_versions = pd.merge(left=df, right=temp.drop(['ver_id_wc', 'ver_id_pub', 'ver_id_auth', 'version_id_raw'], axis=1),
         on=['page_id', 'word_count', 'publish_date', 'authors'],
         how='left')

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 20)

In [ ]:
page106523 = df_versions.query('page_id == 106523')

In [ ]:
df_versions.head()

In [ ]:
df_new = pd.read_csv('../data/aggr_data.csv')

In [ ]:
df_new.head()

In [ ]:
df_new.columns

Look at **1010557**

# Aggregation on a page_id level

Group 1: external_impressions and external_clicks:
+ page_id
+ date

Group 2: video_play, page_impressions, clickouts:
+ page_id
+ date
+ URL
+ Author (edge case)

Group 3: daily_likes, daily_dislikes:
+ page_id
+ date
+ publish_date

In [ ]:
df_new.head()

In [ ]:
url_col = df_new[['page_id', 'date', 'url']]

url_col = url_col[['page_id', 'date', 'url']]\
    .groupby(['page_id', 'date']) \
        .agg(lambda url: ';'.join(url))

url_col = url_col.url.apply(lambda urls: sorted(list(set(urls.split(';')))))
url_col = url_col.apply(','.join).to_frame().reset_index()

In [ ]:
df_agg_url = df_new[['page_id', 'date', 'url', 'video_play', 'page_impressions', 'clickouts', 'version_id_new']]\
                .groupby(['page_id', 'date', 'url']).max()

df_agg_page_date_pubdate =  df_new[['page_id', 'date', 'publish_date', 'daily_likes', 'daily_dislikes']]\
               .groupby(['page_id', 'date', 'publish_date']).sum()

df_agg_page_date = df_new.drop(['publish_date', 'old_index', 'url', 'video_play', 
                                'version_id_new', 'version_id', 'word_count', 'page_impressions', 'clickouts', 
                               'daily_likes', 'daily_dislikes'], axis=1).drop_duplicates()

In [ ]:
df_agg_url.shape, df_agg_page_date_pubdate.shape, df_agg_page_date.shape

In [ ]:
df_agg_page_date.columns

In [ ]:
df_agg_full_0 = pd.merge(left=df_new[['page_id', 'date']].drop_duplicates(), 
                         right=url_col, on=['page_id', 'date'])
df_agg_full_1 = pd.merge(left=df_agg_full_0, right=df_agg_page_date, on=['page_id', 'date'], how='inner')
df_agg_full_2 = pd.merge(left=df_agg_full_1, right=df_agg_page_date_pubdate, on=['page_id', 'date'], how='inner')
df_agg_full_3 = pd.merge(left=df_agg_full_2, right=df_agg_url, on=['page_id', 'date', 'url'], how='inner')

In [ ]:
mgn.matrix(df_agg_full_3)

In [ ]:
df_agg_full = pd.merge(left=df_agg_page_date, right=df_agg_page_date_pubdate, on=['page_id', 'date'], how='inner')
#df_agg_full = pd.merge()

# How frequently the article was published?

How to catch it:
1. Identify "first" dates
    * Perhaps, difference btw dates?
* Min, max, avg distance btw dates
* Number of "first" dates (= peaks)

#

In [1]:
import pandas as pd

df = pd.read_csv('../../data/data_aggr.csv')

In [2]:
import pygwalker as pyg

walker = pyg.walk(df, spec="./page_timelines.json")

Box(children=(HTML(value='<div id="ifr-pyg-000615d67d988b32QYhFCsTKtogM5uZ6" style="height: auto">\n    <head>…